# SmartClip AI - Google Colab

Transform long videos into viral short clips using AI.

**Features:** AI-powered clip detection, Auto captions, Multiple aspect ratios

**Providers:** OpenAI (default), Groq (free), Gemini (free tier), Custom OpenAI-compatible APIs

## Step 1: Install SmartClip AI

In [ ]:
# Install SmartClip AI
!pip install -q git+https://github.com/sakirsyarian/sclip.git
!apt-get update -qq && apt-get install -qq -y ffmpeg
!sclip --check-deps
print('Setup complete!')

## Step 2: Configure API Key

Choose your provider:
- **OpenAI** (default): https://platform.openai.com/api-keys
- **Groq** (free): https://console.groq.com
- **Gemini** (free tier): https://aistudio.google.com/apikey
- **Custom**: Any OpenAI-compatible API (Together AI, OpenRouter, etc.)

In [ ]:
#@title API Configuration
import os
from getpass import getpass

provider = 'openai' #@param ['openai', 'groq', 'gemini', 'custom']
custom_base_url = '' #@param {type:'string'}

# Map provider to environment variable
env_vars = {
    'openai': 'OPENAI_API_KEY',
    'groq': 'GROQ_API_KEY',
    'gemini': 'GEMINI_API_KEY',
    'custom': 'OPENAI_API_KEY'
}

env_var = env_vars[provider]
api_key = None

# Try to load from Colab Secrets
try:
    from google.colab import userdata
    api_key = userdata.get(env_var)
    if api_key:
        print(f'API key loaded from Colab Secrets ({env_var})')
except:
    pass

# Prompt if not found
if not api_key:
    api_key = getpass(f'Enter {provider.upper()} API key: ')

os.environ[env_var] = api_key

# Set custom base URL if provided
if provider == 'custom' and custom_base_url:
    os.environ['OPENAI_BASE_URL'] = custom_base_url
    print(f'Custom base URL: {custom_base_url}')

print(f'Provider: {provider} - API key configured!')

## Step 3: Upload Video

Choose: Upload (max 100MB), YouTube URL, or Google Drive

In [ ]:
#@title Video Source
video_source = 'YouTube URL' #@param ['Upload from computer', 'YouTube URL', 'Google Drive']
youtube_url = '' #@param {type:'string'}
drive_path = '/content/drive/MyDrive/video.mp4' #@param {type:'string'}

VIDEO_PATH = None
YOUTUBE_URL = None

if video_source == 'Upload from computer':
    from google.colab import files
    print('Select video file (max 100MB)...')
    uploaded = files.upload()
    if uploaded:
        VIDEO_PATH = list(uploaded.keys())[0]
        print(f'Uploaded: {VIDEO_PATH}')
elif video_source == 'YouTube URL':
    if youtube_url:
        YOUTUBE_URL = youtube_url
        print(f'YouTube: {YOUTUBE_URL}')
    else:
        print('Enter YouTube URL above')
elif video_source == 'Google Drive':
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    if os.path.exists(drive_path):
        VIDEO_PATH = drive_path
        print(f'Found: {VIDEO_PATH}')
    else:
        print(f'Not found: {drive_path}')

## Step 4: Generate Clips

In [ ]:
#@title Settings
max_clips = 5 #@param {type:'slider', min:1, max:10, step:1}
min_duration = 60 #@param {type:'slider', min:30, max:120, step:15}
max_duration = 180 #@param {type:'slider', min:60, max:300, step:30}
aspect_ratio = '9:16' #@param ['9:16', '1:1', '16:9']
caption_style = 'default' #@param ['default', 'bold', 'minimal', 'karaoke']
language = 'id' #@param ['id', 'en', 'auto']
transcriber_provider = 'openai' #@param ['openai', 'groq', 'local']
analyzer_provider = 'openai' #@param ['openai', 'groq', 'gemini']
custom_model = '' #@param {type:'string'}
dry_run = False #@param {type:'boolean'}

cmd = 'sclip'
if YOUTUBE_URL:
    cmd += f' -u "{YOUTUBE_URL}"'
elif VIDEO_PATH:
    cmd += f' -i "{VIDEO_PATH}"'
else:
    raise Exception('No video! Run Step 3 first.')

cmd += f' -o /content/output -n {max_clips} --min-duration {min_duration} --max-duration {max_duration}'
cmd += f' -a {aspect_ratio} -s {caption_style} -l {language}'
cmd += f' --transcriber {transcriber_provider} --analyzer {analyzer_provider}'

if custom_model:
    cmd += f' --analyzer-model {custom_model}'

if dry_run:
    cmd += ' --dry-run'

print(f'Running: {cmd}')
!{cmd}

## Step 5: Download Clips

In [ ]:
#@title Download
download_method = 'Download to computer' #@param ['Download to computer', 'Save to Google Drive']
drive_folder = '/content/drive/MyDrive/SmartClip_Output' #@param {type:'string'}

import glob, os, shutil
clips = glob.glob('/content/output/*.mp4')

if not clips:
    print('No clips found. Run Step 4 first.')
else:
    print(f'Found {len(clips)} clip(s)')
    if download_method == 'Download to computer':
        from google.colab import files
        for f in clips:
            files.download(f)
        print('Download complete!')
    else:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        os.makedirs(drive_folder, exist_ok=True)
        for f in clips:
            shutil.copy(f, drive_folder)
        print(f'Saved to {drive_folder}')

---

## Tips

- **GPU**: Runtime > Change runtime type > T4 GPU (faster rendering)
- **API Keys**: Save in Colab Secrets (key icon in sidebar) for convenience
- **Large Videos**: Use YouTube URL or Google Drive for files >100MB
- **Preview**: Use dry_run=True to see clips before rendering
- **Free Options**: Use Groq (transcriber + analyzer) or Gemini (analyzer) for free
- **Custom API**: Set provider='custom' and custom_base_url for Together AI, OpenRouter, etc.